In [234]:
import pandas as pd
from openbb import obb
from datetime import date,datetime, timedelta
from dateutil.relativedelta import relativedelta
import numpy as np
import calendar

obb.user.preferences.output_type = 'dataframe'

In [162]:
tickers = ['AAPL', 'NVDA', 'googl']

todays_date = date.today()

yesterday = todays_date - timedelta(1)

five_years = todays_date - relativedelta(years = 5)


In [294]:
days = 365
years = list(range(2020, 2026))
months= list(range(1,13))

def days_in_month(years, months):
    days_dict = {}
    for year in years:
        for month in months:
            days = calendar.monthrange(year, month)[1]
            days_dict[(year, month)] = days
    return days_dict

days_per_month = days_in_month(years, months)
number_of_days = sum(days_per_month.values())
number_of_days

2192

## Simple Returns

$$
\frac{p_1}{p_{t-1}} - 1 
$$

## Compound Return

$$
\log \frac{p_t}{p_{t-1}}
$$

## CAGR (Compound Annual Growth Rate) ￼

$$ 
(\frac{endingvalue}{beginning value})^\frac{1}{n} - 1
$$
###### You want the beginning value to understand the initial starting point of the investment, along with the ending value as the most recent point of the investment

###### It's always good to be able to see how much an investment grows given annual rate, ignoring volatility rate

### Sharpe Ratio

$$
\frac{R_p - R_f}{\sigma_p}
$$


In [296]:
def closing_data(tickers):
    data = []
    dataframe = pd.DataFrame()
    for ticker in tickers:
        # more columns
        dataframe = obb.equity.price.historical(ticker,provider = 'yfinance', start_date = five_years)
        dataframe['simple_returns'] = (dataframe.loc[:,'close'] / dataframe.loc[:,'close'].shift()) - 1
        dataframe['compound_returns'] = np.log((dataframe.loc[:, 'close'] / dataframe.loc[:,'close'].shift()))  
        dataframe['CAGR_Percent'] = (dataframe.iloc[-1]['close'] / dataframe.iloc[0]['close']) ** (1/5) - 1
        dataframe['annualized_volatility'] = ((dataframe.loc[:,'close'] / dataframe.loc[:,'close'].shift())-1).std() * np.sqrt(252)



        
        # low_data = dataframe[['low']].rename({'low': ticker}) 
    return dataframe

closing_data(tickers)    

,open,high,low,close,volume,split_ratio,dividend,simple_returns,compound_returns,CAGR_Percent,annualized_volatility
date,,,,,,,,,,,
2020-08-06,73.807503,75.313499,73.595001,75.247498,33306000,0.0,0.0,NaN,NaN,0.211135,0.31022
2020-08-07,75.452003,76.004501,74.313499,74.918503,27718000,0.0,0.0,-0.004372,-0.004382,0.211135,0.31022
2020-08-10,74.540001,75.357498,73.874496,74.841003,20546000,0.0,0.0,-0.001034,-0.001035,0.211135,0.31022
2020-08-11,74.699997,75.522003,73.909500,74.027000,31098000,0.0,0.0,-0.010876,-0.010936,0.211135,0.31022
2020-08-12,74.356003,75.583504,74.250000,75.362000,22512000,0.0,0.0,0.018034,0.017873,0.211135,0.31022
...,...,...,...,...,...,...,...,...,...,...,...
2025-07-31,195.710007,195.990005,191.089996,191.899994,51329200,0.0,0.0,-0.023559,-0.023841,0.211135,0.31022
2025-08-01,189.029999,190.830002,187.820007,189.130005,34832200,0.0,0.0,-0.014435,-0.014540,0.211135,0.31022
2025-08-04,190.289993,195.270004,190.119995,195.039993,31547400,0.0,0.0,0.031248,0.030770,0.211135,0.31022


In [ ]:
def downloading_option_data(tickers):
    tickers = [ticker.upper() for ticker in tickers]

    hasher = {}
    
    for ticker in tickers:
        dataframe = obb.derivatives.options.chains(ticker, provider="yfinance", start_date = todays_date).dropna()
        hasher[ticker] = dataframe.set_index['expiration', 'strike', 'option_type', 'symbol']
    return hasher
    
    

data = downloading_option_data(tickers)

In [ ]:
data['AAPL']